# Prueba con GPLearn

In [23]:
import pandas as pd
import matplotlib 
from gplearn.genetic import SymbolicRegressor
from gplearn.functions import make_function
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.utils.random import check_random_state
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, r2_score
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
#import graphviz
import pandas as pd
import random
import re

### Coeficiente de arrastre

In [2]:
# cargando datos

# importacion de DF
file_path = 'D:\\CODES\\LIB_SR\\Dataset_coeficientes\\df_cdrag_25.txt'
df_cdrag_25 = pd.read_csv(file_path, delimiter=',')

file_path = 'D:\\CODES\\LIB_SR\\Dataset_coeficientes\\df_cdrag_53.txt'
df_cdrag_53 = pd.read_csv(file_path, delimiter=',')

file_path = 'D:\\CODES\\LIB_SR\\Dataset_coeficientes\\df_cdrag_74.txt'
df_cdrag_74 = pd.read_csv(file_path, delimiter=',')

file_path = 'D:\\CODES\\LIB_SR\\Dataset_coeficientes\\df_cdrag_102.txt'
df_cdrag_102 = pd.read_csv(file_path, delimiter=',')

In [3]:
# definiendo conjuntos de train y test

# definiendo conjunto de train
df_cdrag_train = pd.concat([df_cdrag_25, df_cdrag_74], ignore_index=True)

# separando entre x e y
y_train = df_cdrag_train.drop(columns=['Current','K','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido','n_celda','Rem','colIndex'])
X_train = df_cdrag_train.drop(columns=['Current','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido',
                                    'n_celda','colIndex','cdrag'])


# definiendo conjunto de test

y_test = df_cdrag_53.drop(columns=['Current','K','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido','n_celda','Rem','colIndex'])
X_test = df_cdrag_53.drop(columns=['Current','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido',
                                    'n_celda','colIndex','cdrag'])

In [12]:
X_test

,K,Rem
0,0.6,2824.604561
1,0.6,2824.604561
2,0.6,2352.302465
3,0.6,2000.632597
4,0.6,1878.735604
...,...,...
8605,1.2,9267.885175
8606,1.2,8959.387980
8607,1.2,8691.558442
8608,1.2,8537.950633


In [4]:
# creando funciones auxiliares

# S ^ (-0.6)
def pot1(x1):
    with np.errstate(divide='ignore', invalid='ignore'):
        result = np.where(x1 > 0, np.power(x1, -0.6), 0)
    return result

pot1_fn = make_function(function=pot1, 
                        name='pot1', 
                        arity=1)

# 5*Re ^ (-0.23)
def pot2(x1):
    with np.errstate(divide='ignore', invalid='ignore'):
        result = np.where(x1 > 0, 5*np.power(x1, -0.23), 0)
    return result

pot2_fn = make_function(function=pot2, 
                        name='pot2', 
                        arity=1)



In [48]:
def rewrite(expression):

    # Reemplazar funciones específicas con sus representaciones matemáticas
    expression = re.sub(r"add\(([^,]+), ([^\)]+)\)", r"(\1 + \2)", expression)
    expression = re.sub(r"sub\(([^,]+), ([^\)]+)\)", r"(\1 - \2)", expression)
    expression = re.sub(r"mul\(([^,]+), ([^\)]+)\)", r"(\1 * \2)", expression)
    expression = re.sub(r"div\(([^,]+), ([^\)]+)\)", r"(\1 / \2)", expression)
    expression = re.sub(r"pow\(([^,]+), ([^\)]+)\)", r"\1 ** \2", expression)
    
    # Reemplazar la función pot1
    expression = re.sub(r"pot1\((X\d)\)", r"\1 ** (-0.6)", expression)
    # Reemplazar la función pot2
    expression = re.sub(r"pot2\((X\d)\)", r"5 * \1 ** (-0.23)", expression)
    
    # Reemplazar variables (ajustar según el modelo)
    expression = expression.replace("X0", "S")
    expression = expression.replace("X1", "Re")
    
    return expression


In [5]:
# entrenando modelo 

# Modelo
model = SymbolicRegressor(population_size=5000,
                           generations=20, 
                           stopping_criteria=0.01,
                           p_crossover=0.7, 
                           p_subtree_mutation=0.1,
                           p_hoist_mutation=0.05, 
                           p_point_mutation=0.1,
                           max_samples=0.9, 
                           verbose=1,
                           parsimony_coefficient=0.01, 
                           random_state=123,
                           function_set=['add', 'sub', 'mul', 'div', 'sqrt', 'log',pot1_fn,pot2_fn])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
#print("Expresión:", expression)


d:\env\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    13.80      4.44235e+07        5        0.0727931        0.0695565     12.12m
   1     9.11      3.59451e+07        5        0.0723137        0.0738714      8.17m
   2     6.64          25901.6        5        0.0720671        0.0760905     10.96m
   3     6.11          12859.9        9        0.0698535        0.0740827      8.44m
   4     6.26          37314.4        5        0.0718866        0.0777156      9.97m
   5     5.32          22445.9        8        0.0717459        0.0728612      6.35m
   6     5.20          23145.1        5        0.0717727        0.0787408      7.15m
   7     5.17      1.83698e+07        5         0.071877         0.077802      6.56m
   8     5.18          22152.1        5        0.0718211        0.0783049  

In [49]:
rewrite(expression.__str__())

'(5 * Re ** (-0.23) + S ** (-0.6))'

In [6]:
# Predecir en los datos de prueba
y_pred = model.predict(X_test)


In [10]:
# comparar con modelo de Rafael

def cdrag(S, Re):
    return S**(-0.6) + 5*Re**(-0.23)

def cdrag_gp(S, Re):
    return S**(-0.6) + 5*Re**(-0.23)

In [11]:
# Calcular cdrag_pred y cdrag_gp_pred en vectores separados
cdrag_pred = X_test.apply(lambda row: cdrag(row['K'], row['Rem']), axis=1).values
cdrag_gp_pred = X_test.apply(lambda row: cdrag_gp(row['K'], row['Rem']), axis=1).values

# Calcular R^2 y MSE para cdrag
r2_cdrag = r2_score(y_test['cdrag'], cdrag_pred)
mse_cdrag = mean_squared_error(y_test['cdrag'], cdrag_pred)

# Calcular MAE y MAPE para cdrag
mae_cdrag = mean_absolute_error(y_test['cdrag'], cdrag_pred)
mape_cdrag = mean_absolute_percentage_error(y_test['cdrag'], cdrag_pred)

# Calcular R^2 y MSE para cdrag_gp
r2_cdrag_gp = r2_score(y_test['cdrag'], cdrag_gp_pred)
mse_cdrag_gp = mean_squared_error(y_test['cdrag'], cdrag_gp_pred)

# Calcular MAE y MAPE para cdrag_gp
mae_cdrag_gp = mean_absolute_error(y_test['cdrag'], cdrag_gp_pred)
mape_cdrag_gp = mean_absolute_percentage_error(y_test['cdrag'], cdrag_gp_pred)

#  Mostrar Metricas 

print(f"R^2 cdrag original: {r2_cdrag}, R^2 cdrag GPLearn: {r2_cdrag_gp} ")
print(f"MSE cdrag original: {mse_cdrag}, MSE cdrag GPLearn: {mse_cdrag_gp} ")
print(f"MAE cdrag original: {mae_cdrag}, MAE cdrag GPLearn: {mae_cdrag_gp} ")
print(f"MAPE cdrag original: {mape_cdrag*100}*100, MAPE cdrag GPLearn: {mape_cdrag_gp*100}*100 ")

R^2 cdrag original: 0.8904926780136748, R^2 cdrag GPLearn: 0.8904926780136748 
MSE cdrag original: 0.011588230963563853, MSE cdrag GPLearn: 0.011588230963563853 
MAE cdrag original: 0.07580331737181549, MAE cdrag GPLearn: 0.07580331737181549 
MAPE cdrag original: 0.03744829246296146, MAPE cdrag GPLearn: 0.03744829246296146 


### Prueba 2: Separacion entre laminar y turbulento

In [34]:
df_cdrag_train = pd.concat([df_cdrag_25, df_cdrag_74], ignore_index=True)

# separando entre desplazamiento laminar y turbulento
df_cdrag_train_lam = df_cdrag_train[df_cdrag_train['Rem'] <= 2000]
df_cdrag_train_turb = df_cdrag_train[df_cdrag_train['Rem'] > 2000]

y_train_lam = df_cdrag_train_lam.drop(columns=['Current','K','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido','n_celda','Rem','colIndex'])
X_train_lam = df_cdrag_train_lam.drop(columns=['Current','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido',
                                    'n_celda','colIndex','cdrag'])

y_train_turb = df_cdrag_train_turb.drop(columns=['Current','K','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido','n_celda','Rem','colIndex'])
X_train_turb = df_cdrag_train_turb.drop(columns=['Current','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido',
                                    'n_celda','colIndex','cdrag'])

test_lam = df_cdrag_53[df_cdrag_53['Rem'] <= 2000]
test_turb = df_cdrag_53[df_cdrag_53['Rem'] > 2000]

# definiendo conjunto de test

y_test_lam = test_lam.drop(columns=['Current','K','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido','n_celda','Rem','colIndex'])
X_test_lam = test_lam.drop(columns=['Current','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido',
                                    'n_celda','colIndex','cdrag'])

y_test_turb = test_turb.drop(columns=['Current','K','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido','n_celda','Rem','colIndex'])
X_test_turb = test_turb.drop(columns=['Current','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido',
                                    'n_celda','colIndex','cdrag'])


In [38]:
# S ^ (-0.6)
def pot1(x1):
    with np.errstate(divide='ignore', invalid='ignore'):
        result = np.where(x1 > 0, np.power(x1, -0.6), 0)
    return result

pot1_fn = make_function(function=pot1, 
                        name='pot1', 
                        arity=1)

# 5*Re ^ (-0.23)
def pot2(x1):
    with np.errstate(divide='ignore', invalid='ignore'):
        result = np.where(x1 > 0, 5*np.power(x1, -0.23), 0)
    return result

pot2_fn = make_function(function=pot2, 
                        name='pot2', 
                        arity=1)


In [40]:
# entrenando modelo laminar

# Modelo
model = SymbolicRegressor(population_size=5000,
                           generations=25, 
                           stopping_criteria=0.01,
                           p_crossover=0.7, 
                           p_subtree_mutation=0.1,
                           p_hoist_mutation=0.05, 
                           p_point_mutation=0.1,
                           max_samples=0.9, 
                           verbose=1,
                           parsimony_coefficient=0.01, 
                           random_state=123,
                           function_set=['add', 'sub', 'mul', 'div',pot1_fn,pot2_fn],
                           const_range=(-10, 10))

model.fit(X_train_lam, y_train_lam)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

# Predecir en los datos de prueba
y_pred = model.predict(X_test_lam)

# Calcular métricas
mse = mean_squared_error(y_test_lam, y_pred)
r2 = r2_score(y_test_lam, y_pred)

print("MSE:", mse)
print("R^2:", r2)

d:\env\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    19.46      1.16325e+11       14        0.0989867         0.097607      9.94m
   1    11.90      3.47286e+06       25         0.098282          0.10394      5.99m
   2    10.16      1.99623e+06        9        0.0980644        0.0956614      6.94m
   3     9.37      1.76432e+09        8        0.0961371        0.0935922      6.71m
   4     7.70          11995.3       10          0.09412        0.0958618      5.18m
   5     5.28      2.56837e+06       12        0.0967676        0.0973654      5.82m
   6     5.26           265602        5         0.098622         0.113828      4.71m
   7     5.24          3651.69        6         0.096651        0.0977687      5.83m
   8     5.24          13647.6        4        0.0969893        0.0892327  

In [41]:
# entrenando modelo turbulento

# Modelo
model = SymbolicRegressor(population_size=5000,
                           generations=25, 
                           stopping_criteria=0.01,
                           p_crossover=0.7, 
                           p_subtree_mutation=0.1,
                           p_hoist_mutation=0.05, 
                           p_point_mutation=0.1,
                           max_samples=0.9, 
                           verbose=1,
                           parsimony_coefficient=0.01, 
                           random_state=123,
                           function_set=['add', 'sub', 'mul', 'div',pot1_fn,pot2_fn],
                           const_range=(-10, 10))

model.fit(X_train_turb, y_train_turb)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

# Predecir en los datos de prueba
y_pred = model.predict(X_test_turb)

# Calcular métricas
mse = mean_squared_error(y_test_turb, y_pred)
r2 = r2_score(y_test_turb, y_pred)

print("MSE:", mse)
print("R^2:", r2)

d:\env\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    19.46      8.02502e+12       13        0.0642244        0.0631852     26.53m
   1    11.50      5.34598e+07        5        0.0638259        0.0667703     27.34m
   2     9.58      4.75667e+07        5        0.0635333        0.0694023     28.93m
   3     6.43      4.82039e+07        8        0.0634863        0.0698258     23.54m
   4     5.83          70092.2        5        0.0635609        0.0691547     25.87m
   5     5.26      6.08778e+07        5         0.063477        0.0699095     13.39m
   6     5.26        3.352e+06        5        0.0635552        0.0692055     12.44m
   7     5.24          24680.8        5        0.0634513          0.07014      7.98m
   8     5.24           136728        5        0.0634564        0.0700944  

### Ajuste automatico de Exponencial

In [118]:

def pot(x1, const):
    con1 = x1 > 0
    con2 = (const >= -1) & (const < 0)
    with np.errstate(divide='ignore', invalid='ignore'):
        result = np.where(con1 & con2, np.power(x1,const), 0)
    return result

pot= make_function(function=pot, name='pot', arity=2)

# A * x**B
def pot2(x1):
    con1 = x1 > 0
    # con2 = (B >= -0.24) & (B <= -0.22)
    # con3 = (A >=4.9) & (A <=5.1)
    with np.errstate(divide='ignore', invalid='ignore'):
        result = np.where(con1  , 4 *np.power(x1,-0.2), 0)
    return result

pot2= make_function(function=pot2, name='pot2', arity=1)

In [119]:
# entrenando modelo 

# Modelo
model = SymbolicRegressor(population_size=5000,
                           generations=20, 
                           stopping_criteria=0.01,
                           p_crossover=0.7, 
                           p_subtree_mutation=0.1,
                           p_hoist_mutation=0.05, 
                           p_point_mutation=0.1,
                           max_samples=0.9, 
                           verbose=1,
                           parsimony_coefficient=0.01, 
                           random_state=123,
                           function_set=['add', 'sub', 'mul', 'div',pot, pot2])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

d:\env\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\lucas\AppData\Local\Temp\ipykernel_10684\1057681946.py:5: RuntimeWarning: overflow encountered in power
  result = np.where(con1 & con2, np.power(x1,const), 0)


    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    28.01      2.60336e+14       26         0.177252         0.184235     19.45m
   1    13.71      1.53632e+09       29         0.115348         0.119264     10.96m
   2    12.59      3.67498e+07        6          0.07869        0.0790255      8.48m
   3     7.46      1.82761e+07        6         0.078302         0.081466      7.92m
   4     6.49      3.23428e+07        8        0.0709757         0.069946      4.55m
   5     6.65      3.65579e+07        8         0.070597        0.0731209      4.44m
   6     6.48          37148.9        8        0.0705127          0.07388      5.15m
   7     6.28      6.40493e+09        8        0.0707987        0.0715388      5.04m
   8     6.28          44836.3        6        0.0706076        0.0745433  

In [120]:
# Predecir en los datos de prueba
y_pred = model.predict(X_test)

In [121]:
def cdrag(S, Re):
    return S**(-0.6) + 5*Re**(-0.23)

def cdrag_gp(S, Re):
    return S**(-0.598) + 4*Re**(-0.2)

In [122]:
# Calcular cdrag_pred y cdrag_gp_pred en vectores separados
cdrag_pred = X_test.apply(lambda row: cdrag(row['K'], row['Rem']), axis=1).values
cdrag_gp_pred = X_test.apply(lambda row: cdrag_gp(row['K'], row['Rem']), axis=1).values

# Calcular R^2 y MSE para cdrag
r2_cdrag = r2_score(y_test['cdrag'], cdrag_pred)
mse_cdrag = mean_squared_error(y_test['cdrag'], cdrag_pred)

# Calcular MAE y MAPE para cdrag
mae_cdrag = mean_absolute_error(y_test['cdrag'], cdrag_pred)
mape_cdrag = mean_absolute_percentage_error(y_test['cdrag'], cdrag_pred)

# Calcular R^2 y MSE para cdrag_gp
r2_cdrag_gp = r2_score(y_test['cdrag'], cdrag_gp_pred)
mse_cdrag_gp = mean_squared_error(y_test['cdrag'], cdrag_gp_pred)

# Calcular MAE y MAPE para cdrag_gp
mae_cdrag_gp = mean_absolute_error(y_test['cdrag'], cdrag_gp_pred)
mape_cdrag_gp = mean_absolute_percentage_error(y_test['cdrag'], cdrag_gp_pred)

#  Mostrar Metricas 

print(f"R^2 cdrag original: {r2_cdrag}, R^2 cdrag GPLearn: {r2_cdrag_gp} ")
print(f"MSE cdrag original: {mse_cdrag}, MSE cdrag GPLearn: {mse_cdrag_gp} ")
print(f"MAE cdrag original: {mae_cdrag}, MAE cdrag GPLearn: {mae_cdrag_gp} ")
print(f"MAPE cdrag original: {mape_cdrag*100}*100, MAPE cdrag GPLearn: {mape_cdrag_gp*100}*100 ")

R^2 cdrag original: 0.8904926780136748, R^2 cdrag GPLearn: 0.8905797272246793 
MSE cdrag original: 0.011588230963563853, MSE cdrag GPLearn: 0.011579019283979168 
MAE cdrag original: 0.07580331737181549, MAE cdrag GPLearn: 0.07711476735040544 
MAPE cdrag original: 3.7448292462961463*100, MAPE cdrag GPLearn: 3.850526844671428*100 
